Importanje potrebnih biblioteka.

In [3]:
import pandas as pd
import scipy.stats as stats

Definiranje raspona generacija i ispravno ocjenjivanje deepfake videa.

In [4]:
# gornji broj je exclusive
gen_z = range(15, 30)
gen_y = range(30, 45)
gen_z = range(45, 65)
gen_babyboom = range(65, 80)
gen_silent = range(80, 100)

# 0 - autentičan, 1 - deepfake (3, 7, 9)
deepfake_correct = [0, 0, 1, 0, 0, 0, 1, 0, 1, 0]
deepfake_strings = [f'deepfake_{i}' for i in range (1, 11)]

Učitavanje dataseta.

In [7]:
dataset = pd.read_excel(r'dataset.xlsx')

"Ispravljanje" deepfake columna u 0 (ako ispitanik nije točno odredio video) ili 1 (ispitanik je točno prepoznao pravi/deepfake video). Dodavanje novog columna 'rezultat' gdje se nalazi ukupan rezultat ispitanika (raspon od 0 do 10).

In [12]:
# odgovori za deepfake ispravljeni u bodove 0 ili 1 ovisno jesu li pogodili
corrected = dataset
for i, ds in enumerate(deepfake_strings):
    corrected[ds] = (corrected[ds] == deepfake_correct[i]).astype(int)

# dodan column rezultat sa zbrojem bodova za svakog ispitanika
corrected['rezultat'] = corrected[list(deepfake_strings)].sum(axis=1)

df = pd.DataFrame(corrected, columns=['ispitanik', 'dob',
                                   'deepfake_1', 'deepfake_2', 'deepfake_3', 'deepfake_4', 'deepfake_5',
                                   'deepfake_6', 'deepfake_7', 'deepfake_8', 'deepfake_9', 'deepfake_10', 'rezultat'], )
print(df)

     ispitanik  dob  deepfake_1  deepfake_2  deepfake_3  deepfake_4  \
0            1   48           1           1           1           0   
1            2   25           1           1           1           1   
2            3   23           1           0           1           0   
3            4   23           1           1           1           1   
4            5   23           1           0           1           1   
..         ...  ...         ...         ...         ...         ...   
97          98   69           1           1           0           1   
98          99   29           1           1           1           1   
99         100   64           1           1           0           1   
100        101   53           0           1           1           1   
101        102   37           1           1           1           1   

     deepfake_5  deepfake_6  deepfake_7  deepfake_8  deepfake_9  deepfake_10  \
0             1           0           1           1           1    

Računanje prosječnog rezultata ispitanika i određivanje granične p vrijednosti.

In [13]:
avg_result = corrected['rezultat'].mean()
print('Prosjek rezultata', avg_result)
p_value = 0.05
print('Kao granična p vrijednost se uzela', p_value)

Prosjek rezultata 7.401960784313726
Kao granična p vrijednost se uzela 0.05


Prvi test:
H0 - Sve generacije jednako dobro raspoznaju deepfake videe
H1 - Generacija Z bolje raspoznaje deepfake u odnosu na ostale generacije

In [14]:
t1h0 = 'Sve generacije jednako dobro raspoznaju deepfake videe'
t1h1 = 'Generacija Z bolje raspoznaje deepfake u odnosu na ostale generacije'
print('H0:', t1h0)
print('H1:', t1h1)

# filtriranje gen z generacije
gen_z_data = corrected[corrected['dob'].isin(gen_z)]
t_stat_1, p_value_1 = stats.ttest_1samp(gen_z_data['rezultat'], avg_result)

print('P vrijednost je', p_value_1)
if p_value_1 < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')

H0: Sve generacije jednako dobro raspoznaju deepfake videe
H1: Generacija Z bolje raspoznaje deepfake u odnosu na ostale generacije
P vrijednost je 0.012318727935471592
Odbacuje se H0.


Drugi test:
H0 - Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko prate društvene mreže
H1 - Ispitanici koji društvene mreže prate barem jednom dnevno bolje prepoznaju deepfake videe

In [15]:
t2h0 = 'Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko prate društvene mreže'
t2h1 = 'Ispitanici koji društvene mreže prate barem jednom dnevno bolje prepoznaju deepfake videe'
print('H0:', t2h0)
print('H1:', t2h1)

# filtriranje onih koji mreže prate bar jednom dnevno
social_media_data = corrected[corrected['cesto_mreze'] == 1]
t_stat_2, p_value_2 = stats.ttest_1samp(social_media_data['rezultat'], avg_result)

print('P vrijednost je', p_value_2)
if p_value_2 < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')

H0: Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko prate društvene mreže
H1: Ispitanici koji društvene mreže prate barem jednom dnevno bolje prepoznaju deepfake videe
P vrijednost je 0.4021963645076293
Nije moguće odbaciti H0.


Treći test:
H0 - Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko imaju povjerenja u medije
H1 - Ispitanici koji uopće nemaju povjerenja u medije ne raspoznaju dobro deepfake videe